# 🛒 Shopping App Backend Project
Login Details -
'user1': 'pass1',
'user2': 'pass2',
'admin': 'adminpass'
---

In [1]:
print("🛒 Welcome to the Mohit's Marketplace!")

🛒 Welcome to the Mohit's Marketplace!


In [2]:
# Sample user and admin database
USERS_DB = {'user1': 'pass1', 'user2': 'pass2'}
ADMINS_DB = {'admin': 'adminpass'}

# Category database
CATEGORIES_DB = {
    1: 'Footwear',
    2: 'Clothing',
    3: 'Electronics',
    4: 'Accessories'
}

# Product database
PRODUCTS_DB = {
    101: {'name': 'Boots', 'category_id': 1, 'price': 2500},
    102: {'name': 'Jacket', 'category_id': 2, 'price': 4000},
    103: {'name': 'Smartphone', 'category_id': 3, 'price': 15000},
    104: {'name': 'Cap', 'category_id': 4, 'price': 500}
}

# Session and cart storage
SESSIONS = {}  # session_id -> {"role": "admin" or "user", "username": "xyz"}
CARTS = {}     # session_id -> {product_id: quantity}

In [3]:
import uuid

def login(role='user'):
    print(f"🔐 {role.title()} Login")
    username = input("Username: ")
    password = input("Password: ")

    if role == 'admin':
        if ADMINS_DB.get(username) == password:
            session_id = str(uuid.uuid4())
            SESSIONS[session_id] = {'role': 'admin', 'username': username}
            print(f"✅ Admin login successful. Session ID: {session_id}")
            return session_id
        else:
            print("❌ Invalid admin credentials.")
            return None
    elif role == 'user':
        if USERS_DB.get(username) == password:
            session_id = str(uuid.uuid4())
            SESSIONS[session_id] = {'role': 'user', 'username': username}
            CARTS[session_id] = {}
            print(f"✅ User login successful. Session ID: {session_id}")
            return session_id
        else:
            print("❌ Invalid user credentials.")
            return None

In [4]:
def view_catalog():
    print("\n📦 Product Catalog:")
    for pid, details in PRODUCTS_DB.items():
        category_name = CATEGORIES_DB.get(details['category_id'], "Unknown")
        print(f"ID: {pid} | {details['name']} | ₹{details['price']} | Category: {category_name}")

In [5]:
def add_to_cart(session_id):
    if SESSIONS[session_id]['role'] != 'user':
        print("❌ Only users can access cart.")
        return

    product_id = int(input("Enter product ID to add: "))
    quantity = int(input("Enter quantity: "))

    if product_id in PRODUCTS_DB:
        cart = CARTS.get(session_id, {})
        cart[product_id] = cart.get(product_id, 0) + quantity
        CARTS[session_id] = cart
        print("✅ Item added to cart.")
    else:
        print("❌ Product not found.")


def remove_from_cart(session_id):
    if SESSIONS[session_id]['role'] != 'user':
        print("❌ Only users can access cart.")
        return

    product_id = int(input("Enter product ID to remove: "))
    if product_id in CARTS[session_id]:
        del CARTS[session_id][product_id]
        print("✅ Item removed from cart.")
    else:
        print("❌ Product not in cart.")


def view_cart(session_id):
    if SESSIONS[session_id]['role'] != 'user':
        print("❌ Only users can access cart.")
        return

    cart = CARTS.get(session_id, {})
    if not cart:
        print("🛒 Your cart is empty.")
        return

    total = 0
    print("\n🧾 Cart Items:")
    for pid, qty in cart.items():
        product = PRODUCTS_DB[pid]
        subtotal = product['price'] * qty
        print(f"{product['name']} (x{qty}) = ₹{subtotal}")
        total += subtotal
    print(f"\n💰 Total = ₹{total}")


def checkout(session_id):
    if SESSIONS[session_id]['role'] != 'user':
        print("❌ Only users can checkout.")
        return

    cart = CARTS.get(session_id, {})
    if not cart:
        print("🛒 Your cart is empty.")
        return

    view_cart(session_id)
    print("\n💳 Payment Options:")
    print("1. UPI\n2. Debit Card\n3. Net Banking")
    choice = input("Choose payment method (1/2/3): ")

    if choice in ['1', '2', '3']:
        print("✅ Your order is successfully placed!")
    else:
        print("❌ Invalid payment option.")

In [6]:
def admin_add_product(session_id):
    if SESSIONS[session_id]['role'] != 'admin':
        print("❌ Only admins can add products.")
        return

    pid = int(input("Enter new Product ID: "))
    name = input("Enter product name: ")
    price = int(input("Enter price: "))
    print("Available Categories:")
    for cid, cname in CATEGORIES_DB.items():
        print(f"{cid}: {cname}")
    category_id = int(input("Enter category ID: "))

    if category_id not in CATEGORIES_DB:
        print("❌ Invalid category.")
        return

    PRODUCTS_DB[pid] = {'name': name, 'category_id': category_id, 'price': price}
    print("✅ Product added successfully.")


def admin_update_product(session_id):
    if SESSIONS[session_id]['role'] != 'admin':
        print("❌ Only admins can update products.")
        return

    pid = int(input("Enter Product ID to update: "))
    if pid not in PRODUCTS_DB:
        print("❌ Product not found.")
        return

    name = input("New name (leave blank to skip): ")
    price = input("New price (leave blank to skip): ")
    category_id = input("New category ID (leave blank to skip): ")

    if name:
        PRODUCTS_DB[pid]['name'] = name
    if price:
        PRODUCTS_DB[pid]['price'] = int(price)
    if category_id and int(category_id) in CATEGORIES_DB:
        PRODUCTS_DB[pid]['category_id'] = int(category_id)

    print("✅ Product updated.")


def admin_delete_product(session_id):
    if SESSIONS[session_id]['role'] != 'admin':
        print("❌ Only admins can delete products.")
        return

    pid = int(input("Enter Product ID to delete: "))
    if pid in PRODUCTS_DB:
        del PRODUCTS_DB[pid]
        print("✅ Product deleted.")
    else:
        print("❌ Product not found.")


def admin_add_category(session_id):
    if SESSIONS[session_id]['role'] != 'admin':
        print("❌ Only admins can add categories.")
        return

    cid = int(input("Enter new Category ID: "))
    name = input("Enter Category Name: ")
    CATEGORIES_DB[cid] = name
    print("✅ Category added.")


def admin_delete_category(session_id):
    if SESSIONS[session_id]['role'] != 'admin':
        print("❌ Only admins can delete categories.")
        return

    cid = int(input("Enter Category ID to delete: "))
    if cid in CATEGORIES_DB:
        del CATEGORIES_DB[cid]
        print("✅ Category deleted.")
    else:
        print("❌ Category not found.")

In [7]:
def run_app():
    print("🛒 Welcome to the Mohit's Marketplace!")

    while True:
        print("\n🔑 Choose an option:")
        print("1. User Login\n2. Admin Login\n3. Exit")
        choice = input("Enter choice: ")

        if choice == '1':
            session_id = login('user')
            if session_id:
                user_menu(session_id)
        elif choice == '2':
            session_id = login('admin')
            if session_id:
                admin_menu(session_id)
        elif choice == '3':
            print("👋 Thank you for visiting the Mohit's Marketplace!")
            break
        else:
            print("❌ Invalid choice.")


def user_menu(session_id):
    while True:
        print("\n🧍 User Menu:")
        print("1. View Catalog")
        print("2. Add to Cart")
        print("3. Remove from Cart")
        print("4. View Cart")
        print("5. Checkout")
        print("6. Logout")
        choice = input("Enter choice: ")

        if choice == '1':
            view_catalog()
        elif choice == '2':
            add_to_cart(session_id)
        elif choice == '3':
            remove_from_cart(session_id)
        elif choice == '4':
            view_cart(session_id)
        elif choice == '5':
            checkout(session_id)
        elif choice == '6':
            print("🚪 Logged out.")
            break
        else:
            print("❌ Invalid choice.")


def admin_menu(session_id):
    while True:
        print("\n🧑‍💼 Admin Menu:")
        print("1. View Catalog")
        print("2. Add Product")
        print("3. Update Product")
        print("4. Delete Product")
        print("5. Add Category")
        print("6. Delete Category")
        print("7. Logout")
        choice = input("Enter choice: ")

        if choice == '1':
            view_catalog()
        elif choice == '2':
            admin_add_product(session_id)
        elif choice == '3':
            admin_update_product(session_id)
        elif choice == '4':
            admin_delete_product(session_id)
        elif choice == '5':
            admin_add_category(session_id)
        elif choice == '6':
            admin_delete_category(session_id)
        elif choice == '7':
            print("🚪 Logged out.")
            break
        else:
            print("❌ Invalid choice.")